### Analysis of Lipinski rules in predicted molecules
In this notebook, a sample of random molecules is collected from PubChem and analysed with Drug Predictor's model. Then an analysis is conducted to observes any possible bias in the compliance to Lipinski rules between molecules predicted with high and low probabilities.

#### Imports and loads

In [123]:
import os
import random
import pandas as pd
import numpy as np
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
from compute_fp_note import Compute_FP
from tensorflow.keras.models import load_model


In [124]:
# Training dataset, loaded to avoid grabbing molecules used for training
pubchem = pd.read_csv(os.path.join('..', 'data', '01_raw', 'dataset_pubchem.csv'))

In [125]:
# The fingerprints used to train the model
with open(os.path.join('..', 'data', '05_model_input', 'selected_fp.txt')) as file:
            selected_fp = file.readline()

In [126]:
# Load a computer wich holds all the functions to calculate any type of fingerprint
computer = Compute_FP()

In [127]:
# Load the CNN model
model = load_model(os.path.join('..', 'data', '06_models', 'def_model.hd5'))

#### Collecting a n number of molecules not present in the training dataset

In [128]:
n_mols = 100

In [129]:
labeled_cids = list(pubchem['cid'])
random_cids = []
while len(random_cids) < 10:
    i = random.randint(2, 15000000)
    if i not in labeled_cids:
        random_cids.append(i)

#### Building a dataframe with the selected cids

Functions

In [130]:
def get_smiles(cid):
    try:
        compound = pcp.Compound.from_cid(cid)
        return compound.isomeric_smiles
    except:
        'no smiles found'
        return None

In [131]:
def get_rdkit_molecule(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol
    except:
        print('no molecule found')
        return None

In [132]:
def get_fp(mol):
    return computer.relate_fp_functions(selected_fp, mol)

Building a Dataframe with 'cid', 'smiles' and 'molecule' columns

In [133]:
randoms = pd.DataFrame()

In [134]:
randoms['cid'] = random_cids

In [135]:
randoms['smiles'] = randoms['cid'].map(get_smiles)

In [136]:
randoms['molecule'] = randoms['smiles'].map(get_rdkit_molecule)

In [137]:
randoms['fingerprints'] = randoms['molecule'].map(get_fp)

In [138]:
randoms = randoms.dropna()

In [139]:
randoms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cid           10 non-null     int64 
 1   smiles        10 non-null     object
 2   molecule      10 non-null     object
 3   fingerprints  10 non-null     object
dtypes: int64(1), object(3)
memory usage: 448.0+ bytes


Saving the model for later if n_mols is high

In [140]:
#randoms.to_pickle('randoms.pickle')

#### Predicting the categories

Reshaping the array of fingerprints

In [141]:
fingerprints = np.array(list(randoms['fingerprints']))
reshaped_fps = fingerprints.reshape((fingerprints.shape[0], fingerprints.shape[1], 1))

Obtaining the predictions and the highest probabilities.

In [142]:
probs = model.predict(reshaped_fps)
preds = [np.argmax(x) for x in probs]
max_probs = [np.max(x) for x in probs]

1/1 [==============================] - 0s 103ms/step


Adding predictions and probabilities to the dataframe


In [143]:
randoms['predictions'] = preds

In [144]:
randoms['probabilities'] = max_probs

In [145]:
#randoms.to_pickle('randoms_preds.pickle')

#### Adding Lipinski properties to the dataframe

Functions

In [146]:
def add_lipinski_props(df, molecule_column):
    df['HBondAcceptorCount'] = df[molecule_column].map(Lipinski.NumHAcceptors)
    df['HBondDonorCount'] = df[molecule_column].map(Lipinski.NumHDonors)
    df['MolecularWeight'] = df[molecule_column].map(Descriptors.MolWt)
    df['LogP'] = df[molecule_column].map(Descriptors.MolLogP)

In [147]:
def is_lipinski(x: pd.DataFrame) -> pd.DataFrame:
    """
    Function that applies a set of rules (Lipinski rules) to several columns of a pandas dataframe and returns \
          a dataframe with a new column that states if said rules were passed or not.
    Input: pandas dataframe.
    Output: pandas dataframe.
    """
    # Lipinski rules
    hdonor = x['HBondDonorCount'] < 6
    haccept = x['HBondAcceptorCount'] < 10
    mw = x['MolecularWeight'] < 500
    clogP = x['LogP'] < 5
    # Apply rules to dataframe
    x['RuleFive'] = np.where(((hdonor & haccept & mw) | (hdonor & haccept & clogP) | (hdonor & mw & clogP) | (haccept & mw & clogP)), 1, 0)

In [148]:
add_lipinski_props(randoms, 'molecule')

In [149]:
is_lipinski(randoms)

In [150]:
#randoms.to_pickle(os.path.join('randoms_pred_lip.pickle'))

#### Compare molecules predicted with high and low probabilities

In [151]:
high_prob = randoms[randoms['probabilities']>0.7]
low_prob = randoms[randoms['probabilities']<0.7]

In [152]:
randoms_analysis = randoms.drop(['smiles', 'molecule', 'fingerprints'], axis=1)
high_prob_analysis = high_prob.drop(['smiles', 'molecule', 'fingerprints'], axis=1)
low_prob_analysis = low_prob.drop(['smiles', 'molecule', 'fingerprints'], axis=1)

In [153]:
display(randoms_analysis.groupby('RuleFive')['cid'].count()/len(randoms_analysis))
display(randoms_analysis.groupby('RuleFive').mean())

RuleFive
0    0.2
1    0.8
Name: cid, dtype: float64

,cid,predictions,probabilities,HBondAcceptorCount,HBondDonorCount,MolecularWeight,LogP
RuleFive,,,,,,,
0,12824984.5,9.00,0.702113,10.50,3.00,684.9740,3.455050
1,6972802.0,5.75,0.428089,3.25,0.75,285.6705,3.183913


In [154]:
display(high_prob_analysis.groupby('RuleFive')['cid'].count()/len(high_prob_analysis))
display(high_prob_analysis.groupby('RuleFive').mean())

RuleFive
0    1.0
Name: cid, dtype: float64

,cid,predictions,probabilities,HBondAcceptorCount,HBondDonorCount,MolecularWeight,LogP
RuleFive,,,,,,,
0,10929982.0,8.0,0.942317,11.0,2.0,587.062,1.9662


In [155]:
display(low_prob_analysis.groupby('RuleFive')['cid'].count()/len(low_prob_analysis))
display(low_prob_analysis.groupby('RuleFive').mean())

RuleFive
0    0.111111
1    0.888889
Name: cid, dtype: float64

,cid,predictions,probabilities,HBondAcceptorCount,HBondDonorCount,MolecularWeight,LogP
RuleFive,,,,,,,
0,14719987.0,10.00,0.461910,10.00,4.00,782.8860,4.943900
1,6972802.0,5.75,0.428089,3.25,0.75,285.6705,3.183913


We consistently observed in different datasets that the adherence to the Lipinski rule decreases decreases in the molecules predicted with the highest certainty, which is actually the opposite to the expected. Canonically, a Lipinski complier would be predicted as drugable with higer probability.